In [1]:
# Libraries
import math 
import re
import warnings
import pandas as pd
import numpy  as np

from sklearn.preprocessing   import OneHotEncoder
from sklearn.model_selection import train_test_split

from datetime import datetime as dt

warnings.filterwarnings('ignore')

Lettura dei dataset forniti da Kaggle

In [2]:
# local file paths
dir_name = 'datasets'

fp_properties2016 = dir_name + "/properties_2016.csv"
fp_properties2017 = dir_name + "/properties_2017.csv"
fp_train2016      = dir_name +   "/train_2016_v2.csv"
fp_train2017      = dir_name +      "/train_2017.csv"

In [3]:
# Lettura dei dataframe
df_properties2016 = pd.read_csv(fp_properties2016, low_memory=False)
df_train2016      = pd.read_csv(fp_train2016,      low_memory=False)
df_properties2017 = pd.read_csv(fp_properties2017, low_memory=False)
df_train2017      = pd.read_csv(fp_train2017,      low_memory=False)

In [4]:
# Dimensionalità
print(f'Properites 2016 {df_properties2016.shape}')
print(f'     Train 2016 {     df_train2016.shape}')
print(f'Properites 2017 {df_properties2017.shape}')
print(f'     Train 2017 {     df_train2017.shape}')

Properites 2016 (2985217, 58)
     Train 2016 (90275, 3)
Properites 2017 (2985217, 58)
     Train 2017 (77613, 3)


Non dispongono del log-error di ogni casa, ma solo di quelle che sono state vendute. <br>
Seleziono solo l'insieme di case di cui ho a disposizione il log-error.

Unione in un unico dataset: matengo le sole case di cui conosco il __log-error__ <br>
Se una casa ha più log-error, la colonna è copiata e abbinata a ciascuna data di vendita

In [5]:
# Right-join
df_2016 = pd.merge(df_properties2016, df_train2016, how='right', left_on=['parcelid'], right_on=['parcelid'])
df_2017 = pd.merge(df_properties2017, df_train2017, how='right', left_on=['parcelid'], right_on=['parcelid'])

In [6]:
# Dimensionalità
print(f'Properites 2016 {df_2016.shape}')
print(f'Properites 2017 {df_2017.shape}')

Properites 2016 (90275, 60)
Properites 2017 (77613, 60)


Unisco in un unico dataset i dati del 2016 e del 2017

In [7]:
# Concat
dfAll = pd.concat([df_2016, df_2017], ignore_index=True)

In [8]:
del(df_2016, df_2017)

In [9]:
dfAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167888 entries, 0 to 167887
Data columns (total 60 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   parcelid                      167888 non-null  int64  
 1   airconditioningtypeid         53788 non-null   float64
 2   architecturalstyletypeid      468 non-null     float64
 3   basementsqft                  93 non-null      float64
 4   bathroomcnt                   167854 non-null  float64
 5   bedroomcnt                    167854 non-null  float64
 6   buildingclasstypeid           31 non-null      float64
 7   buildingqualitytypeid         107173 non-null  float64
 8   calculatedbathnbr             166056 non-null  float64
 9   decktypeid                    1272 non-null    float64
 10  finishedfloor1squarefeet      12893 non-null   float64
 11  calculatedfinishedsquarefeet  166992 non-null  float64
 12  finishedsquarefeet12          159519 non-nul

Prima di processare i dati, __casto__ tutti i tipi di dato numerici __da 64 bit__ (tipo di default) __a 32 due bit__ con il doppio scopo di risparmiare la memoria usata e effettuare calcoli più efficienti

In [10]:
# Allegerisco i dati
def int_float_to32(df):
    for c, dtype in zip(df.columns, df.dtypes):
        if dtype == np.float64:        
            df[c] = df[c].astype(np.float32)
        if dtype == np.int64:
            df[c] = df[c].astype(np.int32)
    return df

In [11]:
dfAll = int_float_to32(dfAll)

In [12]:
dfAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167888 entries, 0 to 167887
Data columns (total 60 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   parcelid                      167888 non-null  int32  
 1   airconditioningtypeid         53788 non-null   float32
 2   architecturalstyletypeid      468 non-null     float32
 3   basementsqft                  93 non-null      float32
 4   bathroomcnt                   167854 non-null  float32
 5   bedroomcnt                    167854 non-null  float32
 6   buildingclasstypeid           31 non-null      float32
 7   buildingqualitytypeid         107173 non-null  float32
 8   calculatedbathnbr             166056 non-null  float32
 9   decktypeid                    1272 non-null    float32
 10  finishedfloor1squarefeet      12893 non-null   float32
 11  calculatedfinishedsquarefeet  166992 non-null  float32
 12  finishedsquarefeet12          159519 non-nul

In [13]:
dfAll.shape

(167888, 60)

Guardo quante volte occorrono i parcelid, ossia quante volte una casa è stata venduta tra 2016 e 2017

In [14]:
dfAll.loc[:,'parcelid'].value_counts().head(20)

10857130    3
11991059    3
11842707    3
14010551    3
12478591    3
14672826    3
17164212    3
17237150    3
12612211    3
11186156    2
11991474    2
12273962    2
11061551    2
14659784    2
12467034    2
11266654    2
14008322    2
12752161    2
11887100    2
12239653    2
Name: parcelid, dtype: int64

Una casa è stata venduta al massimo tre volte.
Estraggo le case vendute tre volte tra 2016 e 2017

In [15]:
houses = list(dfAll.loc[:,'parcelid'].value_counts()[dfAll.loc[:,'parcelid'].value_counts() == 3].to_dict().keys())
houses

[10857130,
 11991059,
 11842707,
 14010551,
 12478591,
 14672826,
 17164212,
 17237150,
 12612211]

Ispeziono logerror e data di vendita di queste case

In [16]:
dfAll[dfAll.loc[:,'parcelid'].isin(houses)]\
   .sort_values(by=['parcelid', 'transactiondate']).loc[:, ['parcelid', 'logerror', 'transactiondate']]

,parcelid,logerror,transactiondate
135236,10857130,0.053244,2017-06-09
135237,10857130,0.053244,2017-06-30
135238,10857130,0.290908,2017-08-25
55794,11842707,-0.028400,2016-07-14
55795,11842707,0.057300,2016-08-22
55796,11842707,0.207800,2016-09-29
134115,11991059,2.619876,2017-06-06
134116,11991059,2.670239,2017-06-09
134117,11991059,2.508444,2017-06-13
48461,12478591,0.424000,2016-06-23


Noto che il logerror della stessa casa varia di molto a seconda della data di vendita; nella preprazione dei dati sarà dunque importante prendere in considerazione anche il fattore temporale.

# Train, Validation e Test #

In [17]:
# Given a dataframe and the column-target name,  returns due dataframes:
#    - X with all colummns except for the target
#    - y with the only target column
def split_X_y(df, yname):
    Xnames = list(dfAll.columns)
    Xnames.remove('logerror')
    X = df.loc[:,Xnames]
    y = df.loc[:,yname]
    return X, y

Splitto __X__ e __y__ mantendo in __X__ tutte le colonne ad eccezione del __logerror__, che sarà l'unica colonna di __y__

In [18]:
df_X, df_y = split_X_y(dfAll, 'logerror')

Divisione in __Train__, __Validation__ e __Test__

In [19]:
# Splits the given X and y dataset in three parts:
#    - train      0.8
#    - validation 0.2
#    - test       0.2
def train_validation_test(X, y):
    X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, 
                                                         test_size=0.20, random_state=42)
    X_train, X_val, y_train, y_val  = train_test_split(X_train_80, y_train_80, 
                                                       test_size=0.25, random_state=42)
    return X_train, y_train, X_val, y_val, X_test, y_test

In [20]:
X_train, y_train, X_val, y_val, X_test, y_test = train_validation_test(df_X, df_y)

In [21]:
del(dfAll)

In [22]:
# Prints shape of X_train, X_val and X_test
# If y flag is on, also prints y'shapes
def dimensionality(y=False):
    print(f'X_train {  X_train.shape}')
    print(f'X_val   {    X_val.shape}')
    print(f'X_test  {   X_test.shape}')
    if y:
        print(f'y_train { y_train.shape}')
        print(f'y_val   {   y_val.shape}')
        print(f'y_test  {  y_test.shape}')

In [23]:
dimensionality(y=True)

X_train (100732, 59)
X_val   (33578, 59)
X_test  (33578, 59)
y_train (100732,)
y_val   (33578,)
y_test  (33578,)


In [24]:
X_train.loc[: , ['parcelid', 'transactiondate']].head()

,parcelid,transactiondate
153597,14217523,2017-08-02
146235,11199964,2017-07-11
25650,12627031,2016-04-15
122564,13992985,2017-05-02
84846,12086693,2016-10-13


In [25]:
y_train.head()

153597    0.057681
146235   -0.010815
25650     0.020800
122564    0.001967
84846    -0.020200
Name: logerror, dtype: float32

Ripristino il numero di riga

In [26]:
# Given a dataframe set its rows in range from 0 to n in ascending order
def arange_rows(df):
    df.index = np.arange(len(df))
    return df

In [27]:
for df in [X_train, X_val, X_test, y_train, y_val, y_test]:
    df = arange_rows(df)

In [28]:
X_train.loc[: , ['parcelid', 'transactiondate']].head()

,parcelid,transactiondate
0,14217523,2017-08-02
1,11199964,2017-07-11
2,12627031,2016-04-15
3,13992985,2017-05-02
4,12086693,2016-10-13


In [29]:
y_train.head()

0    0.057681
1   -0.010815
2    0.020800
3    0.001967
4   -0.020200
Name: logerror, dtype: float32

# Preparazione dei dati #

In [30]:
X_train.loc[:,'regionidcounty'].unique()

array([1286., 3101., 2061.,   nan], dtype=float32)

In [31]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33578 entries, 0 to 33577
Data columns (total 59 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      33578 non-null  int32  
 1   airconditioningtypeid         10753 non-null  float32
 2   architecturalstyletypeid      94 non-null     float32
 3   basementsqft                  19 non-null     float32
 4   bathroomcnt                   33572 non-null  float32
 5   bedroomcnt                    33572 non-null  float32
 6   buildingclasstypeid           5 non-null      float32
 7   buildingqualitytypeid         21486 non-null  float32
 8   calculatedbathnbr             33180 non-null  float32
 9   decktypeid                    278 non-null    float32
 10  finishedfloor1squarefeet      2551 non-null   float32
 11  calculatedfinishedsquarefeet  33390 non-null  float32
 12  finishedsquarefeet12          31874 non-null  float32
 13  f

## Rappresentazione non corretta dei Nan ##

Alcune feature rappresentano la assenza di una caratteristica con un Nan, quando in realtà ai fini di algoritmi di Machine Learning sarebbe più opportuno rappresentarle con zero o con False

- __fireplaceflag__ ha valori Nan e True, conversione in una variabile binaria
- __fireplacecnt__ e __poolcnt__ hanno un valore numerico se l'elmento è presente, Nan se non è presente. Rappresentazione dell'assenza da Nan a 0.

In [32]:
# Given a dataframe and a column name, column's values are set to zero if Nan, one otherwise
def set_zero_one(df, col_names):
    for col_name in col_names:
        is_na = df.loc[:,col_name].isna()
        df.loc[:,col_name][is_na]  = 0.
        df.loc[:,col_name][~is_na] = 1.
    return df

In [33]:
# Given a dataframe and a column name, valuse of that column are set to zero if Nan
def nan_to_zero(df, col_names):
    for col_name in col_names:
        df.loc[:,col_name].fillna(0., inplace=True)
    return df

In [34]:
for X in [X_train, X_val, X_test]:
    X = set_zero_one(X, ['fireplaceflag'])
    X = nan_to_zero(X, ['fireplacecnt', 'poolcnt'])

## Rimozione degli Outlier ##

Rimuovo valori estremi di y

In [35]:
# Given X and y dataframe remove all rows which target value is under the first or over the last percentile
def remove_outlier(X, y):
    out1 = y < np.percentile(y, 99.5)
    out2 = y > np.percentile(y, 00.5)
    out  = list(map(lambda o1, o2: o1 and o2, out1, out2))
    X = X[out]
    y = y[out]
    return X, y

In [36]:
dimensionality()

X_train (100732, 59)
X_val   (33578, 59)
X_test  (33578, 59)


In [37]:
X_train, y_train = remove_outlier(X_train, y_train)

In [38]:
dimensionality(y=True)

X_train (99724, 59)
X_val   (33578, 59)
X_test  (33578, 59)
y_train (99724,)
y_val   (33578,)
y_test  (33578,)


## Rimozione colonne con alta percentuale di Nan ##

Rimuovo le colonne con un'alta percentuale di valori assenti, in quanto arricchiscono l'informazione del dataset in maniera molto limitata

In [39]:
# Given the dataframe and the name of a column returns the column
def get_col(df, colName):
    return df.loc[:, colName]

# Given a column returns Nan-count and Nan-percentage
def get_col_nan_info(col):
    count = col.isna().sum()
    tot = len(col)
    perc = count/tot
    return count, perc

# Given the df and a cut-off returns a list of column names with Nan-percentage greater or equal to the cut-off
def get_cols_over_nan_percentage(df, cutoff):
    names = df.columns
    overPercentage = []
    for name in names:
        col = get_col(df, name)
        _ , perc = get_col_nan_info(col)
        if perc > cutoff:
            overPercentage.append(name)
    return overPercentage

In [40]:
col_to_delete = get_cols_over_nan_percentage(X_train, 0.6)

for o in col_to_delete:
    print(f'{o} : {get_col_nan_info(get_col(X_train, o))}')
print(f'Length: {len(col_to_delete)}')

airconditioningtypeid : (67655, 0.6784224459508242)
architecturalstyletypeid : (99455, 0.9973025550519433)
basementsqft : (99676, 0.9995186715334323)
buildingclasstypeid : (99708, 0.9998395571778108)
decktypeid : (99016, 0.992900405118126)
finishedfloor1squarefeet : (91940, 0.9219445670049337)
finishedsquarefeet13 : (99676, 0.9995186715334323)
finishedsquarefeet15 : (95882, 0.9614736673218082)
finishedsquarefeet50 : (91940, 0.9219445670049337)
finishedsquarefeet6 : (99264, 0.9953872688620593)
garagecarcnt : (66611, 0.6679535518029762)
garagetotalsqft : (66611, 0.6679535518029762)
hashottuborspa : (97394, 0.9766355140186916)
poolsizesum : (98619, 0.9889194175925554)
pooltypeid10 : (98711, 0.9898419638201436)
pooltypeid2 : (98407, 0.986793550198548)
pooltypeid7 : (80817, 0.8104067225542497)
regionidneighborhood : (59904, 0.6006979262765232)
storytypeid : (99676, 0.9995186715334323)
threequarterbathnbr : (86493, 0.8673238137258834)
typeconstructiontypeid : (99417, 0.9969215033492439)
yard

In [41]:
# Given a dataframe and a column name returns the dataframe within that column
def remove_column(df, col_names):
    df.drop(col_names, axis=1, inplace=True)
    return df

In [42]:
col_to_delete

['airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'buildingclasstypeid',
 'decktypeid',
 'finishedfloor1squarefeet',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'regionidneighborhood',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'numberofstories',
 'taxdelinquencyflag',
 'taxdelinquencyyear']

In [43]:
for X in [X_train, X_val, X_test]:
    X = remove_column(X, col_to_delete)

In [44]:
dimensionality()

X_train (99724, 33)
X_val   (33578, 33)
X_test  (33578, 33)


## Feature con stesso significato ##

__fireplaceflag__ e __fireplacecnt__: la prima dice se esiste almeno un impianto, la seconda quanti ce ne sono. La seconda feature porta una informazione almeno uguale a quello della prima.

In [45]:
X_train.loc[:,['fireplacecnt', 'fireplaceflag']].head(20)

,fireplacecnt,fireplaceflag
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,1.0,0.0
6,1.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


Non c'è coerenza tra le due feature

In [46]:
sum((get_col(X_train,['fireplacecnt'])[X_train.loc[:,'fireplaceflag'] == 0] > 0).values.ravel())

10773

In [47]:
sum((get_col(X_train,['fireplacecnt'])[X_train.loc[:,'fireplaceflag'] == 1] == 0).values.ravel())

229

In 10 mila osservazioni in cui il flag dice che non ci sono impianti, se ne conta almeno uno

In 200 osservazioni in cui il flag dice che è presente un impianto non se ne contano
E in egual misura in 200 case dove non si contano impianti il flag ne segnala la presenza

Scelgo di mantenere l'informazione portata da __fireplacecnt__ perché più esaustiva

In [48]:
for X in [X_train, X_val, X_test]:
    X = remove_column(X, ['fireplaceflag'])

In [49]:
dimensionality()

X_train (99724, 32)
X_val   (33578, 32)
X_test  (33578, 32)


 <br><br>

Conteggio dei bagni

In [50]:
X_train.loc[:,['fullbathcnt','bathroomcnt']]

,fullbathcnt,bathroomcnt
0,2.0,2.5
1,3.0,3.0
2,2.0,2.0
3,2.0,2.0
4,1.0,1.0
...,...,...
100727,2.0,2.0
100728,2.0,2.0
100729,2.0,2.5
100730,2.0,2.5


In [51]:
sum((get_col(X_train, 'bathroomcnt') - get_col(X_train, 'fullbathcnt') > 1).values.ravel())

11

In solo una decina di istanze il dato non ha lo stessa parte intera

In [52]:
sum((get_col(X_train, 'bathroomcnt') - get_col(X_train, 'fullbathcnt') > 1.5).values.ravel())

1

E in solo una è maggiore di 1.5

Scelgo di mantenere solo la colonna bathroomcnt poiché più dettagliata

In [53]:
for X in [X_train, X_val, X_test]:
    X = remove_column(X, 'fullbathcnt')

In [54]:
dimensionality()

X_train (99724, 31)
X_val   (33578, 31)
X_test  (33578, 31)


<br> <br>

__fips__ e __censurtackblock__

In [55]:
get_col(X_train, ['fips', 'censustractandblock'])

,fips,censustractandblock
0,6059.0,6.059022e+13
1,6037.0,6.037910e+13
2,6037.0,6.037294e+13
3,6059.0,6.059087e+13
4,6037.0,6.037302e+13
...,...,...
100727,6037.0,6.037571e+13
100728,6059.0,6.059089e+13
100729,6111.0,6.111007e+13
100730,6059.0,6.059022e+13


Rappresentano lo stesso dato con diversa rappresentazione numerica

In [56]:
equal = []
for i, j in zip(
        get_col(X_train, ['fips']).values.ravel(),\
        (get_col(X_train, ['censustractandblock']) / 10**10).fillna(0).astype('int32').astype('float32').values.ravel()\
    ):
    equal.append(i==j)
    
len(equal)

99724

In equal ho tutte le colonne per cui la relazione numerica vale, controllo dove non vale

In [57]:
get_col(X_train, ['fips', 'censustractandblock'])[[not e for e in equal]] 

,fips,censustractandblock
301,6059.0,NaN
465,6037.0,NaN
727,6037.0,NaN
804,6037.0,NaN
1021,6059.0,NaN
...,...,...
99663,6037.0,NaN
99733,6037.0,NaN
100174,6037.0,NaN
100179,6037.0,NaN


In [58]:
sum((get_col(X_train, 'censustractandblock')[[not e for e in equal]]).isna())

481

Per i valori in cui non concide che censustractandblock ha valore Nan

Rimuovo la colonna censutractandblock che presenza l'assenza di qualche valore a differenza di fips

In [59]:
for X in [X_train, X_val, X_test]:
    X = remove_column(X, 'censustractandblock')

In [60]:
dimensionality()

X_train (99724, 30)
X_val   (33578, 30)
X_test  (33578, 30)


## Rimozione righe con molti Nan ##

Rimuovo istanze poco significative

In [61]:
# Given the dataframe and the index of the row returns the row
def get_row(df, index):
    return df.loc[index, :]

# Given a row returns Nan-count and Nan-percentage
def get_row_nan_info(row):
    count = row.isna().sum()
    tot = len(row)
    perc = count/tot
    return count, perc

# Given the df and a cut-off returns a list of row ids with Nan-percentage greater or equal the cut-off
def get_rows_over_nan_percentage(df, cutoff):
    overPercentage_indexes = []
    for i in df.index:
        row = get_row(df, i)
        _ , perc = get_row_nan_info(row)
        if perc > cutoff:
            overPercentage_indexes.append(i)
    return overPercentage_indexes

In [62]:
# Given X and y dataframe and a cut-off removes from both all rows whith a percentage of Nan greater than the cut-off
def drop_fullnan_rows(df, dfy, cutoff):
    indexes = get_rows_over_nan_percentage(df, cutoff)
    return df.drop(indexes, axis=0, inplace=True), dfy.drop(indexes, axis=0, inplace=True)

In [63]:
dimensionality(y=True)

X_train (99724, 30)
X_val   (33578, 30)
X_test  (33578, 30)
y_train (99724,)
y_val   (33578,)
y_test  (33578,)


In [64]:
for X, y in [[X_train, y_train], [X_val, y_val], [X_test, y_test]]:
    X, y = drop_fullnan_rows(X, y, 0.5)

In [65]:
dimensionality(y=True)

X_train (99709, 30)
X_val   (33572, 30)
X_test  (33567, 30)
y_train (99709,)
y_val   (33572,)
y_test  (33567,)


Sono state rimosse qualche centinaia di righe che portavano un'informazione ridotta

## Conversione di valor non numerici ##

In [66]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99709 entries, 0 to 100731
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      99709 non-null  int32  
 1   bathroomcnt                   99709 non-null  float32
 2   bedroomcnt                    99709 non-null  float32
 3   buildingqualitytypeid         63558 non-null  float32
 4   calculatedbathnbr             98697 non-null  float32
 5   calculatedfinishedsquarefeet  99224 non-null  float32
 6   finishedsquarefeet12          94874 non-null  float32
 7   fips                          99709 non-null  float32
 8   fireplacecnt                  99709 non-null  float32
 9   heatingorsystemtypeid         62661 non-null  float32
 10  latitude                      99709 non-null  float32
 11  longitude                     99709 non-null  float32
 12  lotsizesquarefeet             88847 non-null  float32
 13  

In [67]:
# Given a dataframe returns al list of column-names which type is different from int32 and float32
def get_not_numeric_cols(df):
    
    def is_numeric(value):
        return value != np.int32 and\
               value != np.float32
    
    not_numeric = []
    for k, v in dict(df.dtypes).items():
        if(is_numeric(v)):
            not_numeric.append(k)
    return not_numeric

In [68]:
not_numeric = get_not_numeric_cols(X_train)
print(*not_numeric, sep='\n')

propertycountylandusecode
propertyzoningdesc
transactiondate


Ho tre valori non numerici

In [69]:
# propertycountylandusecode
values = get_col(X_train, 'propertycountylandusecode')
print(f'Values:\n{values.unique()      }\n')
print(f'Occurcences:\n{values.value_counts()}')

Values:
['122' '0100' '010E' '1111' '34' '010C' '1129' '0101' '1' '010D' '0300'
 '1128' '012C' '01DC' '0200' '1117' '010H' '010F' '0104' '1110' '0108'
 '010V' '0400' '020G' '0103' '010G' '96' '0201' '1210' '01HC' '135' '38'
 '070D' '1222' '010M' '0700' '1014' '0131' '1410' '0109' '1116' '012D'
 '100V' '1720' '1112' '73' '0102' '01HE' '1722' '0110' '0111' '0113'
 '012E' '105' '1420' '040V' '1310' '0301' '6050' '030G' '0114' '0401'
 '0130' '0115' '8800' '1432' '020E' '1012' '1321' '0303' '0105' '1011'
 '1120' '1421' '0' '040B' '1333' '0141' '0204']

Occurcences:
0100    34060
122     16979
010C    11428
0101     8196
34       6396
        ...  
1012        1
6050        1
0115        1
1011        1
0105        1
Name: propertycountylandusecode, Length: 79, dtype: int64


In [70]:
# propertyzoningdesc
values = get_col(X_train, 'propertyzoningdesc')
print(f'Values:\n{values.unique()      }\n')
print(f'Occurcences:\n{values.value_counts()}')

Values:
[nan 'PDR1*' 'LAR1' ... 'BRR1*' 'LVPID*' 'LC105*']

Occurcences:
LAR1          8508
LAR3          3131
LARS          1720
LBR1N         1600
LARD1.5       1442
              ... 
CARS-RM25U       1
LCA1**           1
NORI YY          1
PRRE             1
LBR4N*           1
Name: propertyzoningdesc, Length: 2052, dtype: int64


Per queste due feature mantengo solo le cinque più frequenti: <br>
Copro comunque l'informazione sulla maggioranza della popalazione, senza far esplodere il numero di colonne con il one-hot encoding.

In [71]:
# Given a dataframe, a column-name and a importcance treshold, returns a list of his most frequent values
def get_frequent(df, col_name, important):
    col = get_col(df, col_name)
    names = list(col.value_counts().to_dict().keys())[:important]
    return names

# Given the train-dataframe, some column-names, and importance treshold and a list of dataframes,
# foreach dataframe set a value of the specified columns to 'rare' if is not frequent.
def set_rare(df, col_name, important, dfs):
    frequent = get_frequent(df, col_name, important)
    for d in dfs:
        d.loc[:,col_name][~d.loc[:,col_name].isin(frequent)] = 'rare'
    

In [72]:
for col in ['propertycountylandusecode', 'propertyzoningdesc']:
    set_rare(X_train, col, 5, [X_train, X_val, X_test])

Verifico se ho settatto correttamente l'etichetta *rare*

In [73]:
values = get_col(X_train, 'propertycountylandusecode')
print(f'Values:\n{values.unique()      }\n')
print(f'Occurcences:\n{values.value_counts()}')

Values:
['122' '0100' 'rare' '34' '010C' '0101']

Occurcences:
0100    34060
rare    22650
122     16979
010C    11428
0101     8196
34       6396
Name: propertycountylandusecode, dtype: int64


L'operazione sembra essere avvenuta correttamente :)

In [74]:
# transactiondate
values = get_col(X_train, 'transactiondate')
print(f'Values:\n{values.unique()}\n')
print(f'Occurcences:\n{values.value_counts().head(10)}')

Values:
['2017-08-02' '2017-07-11' '2016-04-15' '2017-05-02' '2016-10-13'
 '2016-06-13' '2017-01-03' '2016-08-15' '2016-10-20' '2016-08-11'
 '2016-05-27' '2017-08-10' '2017-06-09' '2017-05-16' '2016-08-03'
 '2017-01-30' '2017-05-12' '2017-06-28' '2016-07-28' '2016-09-02'
 '2016-03-11' '2017-06-06' '2017-09-08' '2017-09-01' '2016-07-13'
 '2017-07-27' '2017-04-05' '2016-12-07' '2017-05-05' '2016-06-30'
 '2017-01-27' '2016-07-21' '2016-10-07' '2017-07-31' '2017-07-24'
 '2016-05-24' '2016-04-22' '2017-06-29' '2016-04-13' '2017-03-16'
 '2017-06-26' '2016-08-25' '2017-07-14' '2016-09-15' '2017-07-13'
 '2016-03-18' '2016-07-07' '2017-01-05' '2017-01-25' '2017-07-28'
 '2017-02-28' '2017-08-04' '2016-08-29' '2017-04-12' '2016-06-23'
 '2017-09-06' '2016-01-14' '2017-06-02' '2016-08-31' '2016-06-24'
 '2017-05-30' '2017-02-24' '2017-06-08' '2017-06-16' '2017-09-14'
 '2016-02-28' '2017-04-20' '2016-09-20' '2016-04-21' '2017-03-31'
 '2017-03-08' '2017-06-01' '2016-02-08' '2016-03-02' '2017-05-24'
 '

È interessante notare come la maggior parte delle transizioni si concentrino negli ultimi giorni del mese

Trasformo i dati come giorni passati dall' 1 Gennaio di quell'anno

In [75]:
# Add a row to the given dataframe which value is the days passed from the 1th Genuary of that year
def date_to_int(df):
     
    def string_to_date(date_str):
        return dt.strptime(date_str.replace('-', '/'), '%Y/%m/%d')
        
    start = string_to_date('2016-01-01')
    df.loc[:,'int_transactiondate'] = pd.to_datetime(df.loc[:,'transactiondate'], format='%Y/%m/%d')
    df.loc[:,'int_transactiondate'] = (df.loc[:,'int_transactiondate'] - start).astype('timedelta64[D]') % 365

    return df

In [76]:
for X in [X_train, X_val, X_test]:
    X = date_to_int(X)

In [77]:
X_train.loc[:,['parcelid', 'transactiondate', 'int_transactiondate']].head(20)

,parcelid,transactiondate,int_transactiondate
0,14217523,2017-08-02,214.0
1,11199964,2017-07-11,192.0
2,12627031,2016-04-15,105.0
3,13992985,2017-05-02,122.0
4,12086693,2016-10-13,286.0
5,14152642,2016-06-13,164.0
6,17210760,2017-01-03,3.0
7,14145516,2016-08-15,227.0
8,14696645,2016-10-20,293.0
9,11697737,2016-08-11,223.0


# Aggiunta di colonne #

## Media delle transazioni di quel periodo in quella nazione ##

Aggiungo colonna: prezzo medio di vendita per quel mese e anno in quello stato

In [78]:
# Given a dataframe, a month, a year and a region_id returns all rows which match with the params
def get_prices(df, month, year, region_id):
    list_of_dates = list(pd.to_datetime(df.loc[:,'transactiondate'], format='%Y/%m/%d').to_dict().values())
    cond1 = list(df['regionidcounty'] == region_id)
    cond2 = list(map(lambda date: date.year  ==  year, list_of_dates))
    cond3 = list(map(lambda date: date.month == month, list_of_dates))
    cond =  list(map(lambda c1, c2, c3: c1 and c2 and c3, cond1, cond2, cond3))
    return df[cond]

# Given a dataframe and its target, a monthm a year and a region id,
# returns the mean log-error for rows that match with the given params
def get_period_mean(df, dfy, month, year, region_id):
    ret = get_prices(df, month, year, region_id)
    indexes = ret.index
    return dfy[indexes].mean()

# Given country-id, year and month returns the corresponding key for the dictionary
def generate_key(c, y, m):
    return f'{int(c)}_{int(y)}_{int(m)}'

# Given a dataframe and its target, country-ids and years returns a dictionary
# which reprent the mean price for the specified country-id, year and month
def get_dictionary(df, dfy, country_ids, years):
    d = {}
    months = range(1,13)
    for c_id in country_ids:
        for year in years:
            for month in months:
                key = generate_key(c_id, year, month)
                d[key] = get_period_mean(df, dfy, month, year, c_id)
    return d

# Given a list of dataframes and train dataset and its target, country-ids and years
# add a column which values correpond to the mean log-error of sale for that country in that year and month
def add_mean_logerror_column(df_list, dfx, dfy, country_ids, years):
    prices_dict = get_dictionary(dfx, dfy, country_ids, years)
    for df in df_list:
        df = add_mean_logerror_column_aux(df, prices_dict)
    
def add_mean_logerror_column_aux(df, prices_dict):
    rows_null = []
    df['period_mean_price'] = np.nan
    for i in df.index:
        c_id = df.at[i,'regionidcounty']
        month = pd.to_datetime(df.at[i, 'transactiondate'], format='%Y/%m/%d').month
        year  = pd.to_datetime(df.at[i, 'transactiondate'], format='%Y/%m/%d').year
        key = generate_key(c_id, year, month)
        df.at[i, 'period_mean_price'] = prices_dict[key]

In [79]:
add_mean_logerror_column([X_train, X_test, X_val], X_train, y_train, [1286, 3101, 2061], [2016, 2017])

L'informazione transactiondate è stata converita in intero e ne è stata ricavata un'ulteriore informazione. Ora posso rimuovere la colonna object con la data di vendita.

In [80]:
X_train.loc[:,['parcelid', 'regionidcounty', 'transactiondate', 'int_transactiondate', 'period_mean_price']].head(20)

,parcelid,regionidcounty,transactiondate,int_transactiondate,period_mean_price
0,14217523,1286.0,2017-08-02,214.0,0.018485
1,11199964,3101.0,2017-07-11,192.0,0.011598
2,12627031,3101.0,2016-04-15,105.0,0.005537
3,13992985,1286.0,2017-05-02,122.0,0.010939
4,12086693,3101.0,2016-10-13,286.0,0.015765
5,14152642,1286.0,2016-06-13,164.0,0.008718
6,17210760,2061.0,2017-01-03,3.0,0.020255
7,14145516,1286.0,2016-08-15,227.0,0.012602
8,14696645,1286.0,2016-10-20,293.0,0.022827
9,11697737,3101.0,2016-08-11,223.0,0.008038


In [81]:
for X in [X_train, X_val, X_test]:
    X = remove_column(X, ['transactiondate'])

## Media delle transazioni delle case vicine ##

La longitudine ha sempre valori negativi, ne effettuo una trasformazione in valore assoluto per semplificare i conti pur mantendendo la stessa informazione

In [82]:
# Given a dataframe set its longitude to positive
def abs_longitude(df):
    df.loc[:,'longitude'] = abs(df.loc[:,'longitude'])
    return df

In [83]:
for X in [X_train, X_val, X_test]:
     X = abs_longitude(X)

In [84]:
# Given a dataframe, longitude, latitude and distance returns all rows which distance is less or equal to the given distance
def get_neighborhood(df, lon, lat, distance):
    cond1 = lon - distance <= df.loc[:,'longitude']
    cond2 = df.loc[:,'longitude'] <= lon + distance
    cond3 = lat - distance <= df.loc[:,'latitude']
    cond4 = df.loc[:,'latitude'] <= lat + distance
    cond =  list(map(lambda c1, c2, c3, c4: c1 and c2 and c3 and c4, cond1, cond2, cond3, cond4))
    return df[cond]

def get_neighborhood_mean(df, dfy, lat, lon, distance):
    ret = get_neighborhood(df, lat, lon, distance)
    indexes = ret.index
    return dfy[indexes].mean()

In [85]:
# Floor a in_value in a multiple of the distance
def round_lat_lon(dim, dist):
        return dim - (dim % dist)

In [86]:
# Given longitude, latitude and distance generate the corresponding key flooring two values
def generate_key(lon, lat, dist):
    lon = round_lat_lon(lon, dist)
    lat = round_lat_lon(lat, dist)
    key = f'{lon}_{lat}'
    return key

In [87]:
# Generate a dictionary with the mean log-error of the neighborhood, defined by the given distance.
def create_distance_dict(df, dfy, dist):
    
    lon_start = round_lat_lon(df.loc[:,'longitude'].min(), dist)
    lon_end   = round_lat_lon(df.loc[:,'longitude'].max(), dist)
    lat_start = round_lat_lon(df.loc[:,'latitude' ].min(), dist)
    lat_end   = round_lat_lon(df.loc[:,'latitude' ].max(), dist)
    
    dict_dist = {}
    
    lon = lon_start
    lat = lat_start
    
    while(lon < lon_end):
        lat = lat_start
        while(lat < lat_end):
            key = generate_key(lon, lat, dist)
            dict_dist[key] = get_neighborhood_mean(df, dfy, lon, lat, dist)
            lat += dist
        lon += dist
    
    return dict_dist
    

In [88]:
def add_neighborhood_logerror_column_aux(df, dist, dict_dist):
    
        df['neighborhood_mean_price'] = np.nan

        for i in df.index:
            lon = df.at[i,'longitude']
            lat = df.at[i,'latitude']
            key = generate_key(lon, lat, dist)
            try:
                df.at[i, 'neighborhood_mean_price'] = dict_dist[key]
            except:
                print(f'Chiave {key} non esiste')
                pass   
            
        return df

# Given a list of dataframes, train datataset and its target and a distance
# add a column with the mean-price of the neighborhood defined from the distance given
def add_neighborhood_logerror_column(df_list, dfx, dfy, distance):

    dict_dist = create_distance_dict(
        dfx,
        dfy,
        distance
    )
    
    for df in df_list:
        df = add_neighborhood_logerror_column_aux(df, distance, dict_dist)

In [89]:
DIST = 30000

In [90]:
add_neighborhood_logerror_column([X_train, X_val, X_test], X_train, y_train, DIST)

Chiave 118260000.0_34800000.0 non esiste
Chiave 118230000.0_34800000.0 non esiste
Chiave 119430000.0_34380000.0 non esiste
Chiave 119430000.0_34350000.0 non esiste
Chiave 119430000.0_34350000.0 non esiste
Chiave 118230000.0_34800000.0 non esiste
Chiave 118230000.0_34800000.0 non esiste
Chiave 119430000.0_34350000.0 non esiste
Chiave 118260000.0_34800000.0 non esiste
Chiave 119430000.0_34350000.0 non esiste
Chiave 119430000.0_34350000.0 non esiste
Chiave 119430000.0_34350000.0 non esiste
Chiave 118230000.0_34800000.0 non esiste
Chiave 118320000.0_34800000.0 non esiste
Chiave 118260000.0_34800000.0 non esiste
Chiave 119430000.0_34350000.0 non esiste
Chiave 118230000.0_34800000.0 non esiste
Chiave 119460000.0_34350000.0 non esiste
Chiave 118410000.0_34800000.0 non esiste
Chiave 119430000.0_34350000.0 non esiste


Alcuni valori non coperti dal Train, in Validation e Test avranno valore Nan

In [91]:
for X in [X_train, X_val, X_test]:
    sum(X.loc[:,'neighborhood_mean_price'].isna())

In [92]:
X_train.loc[:,['parcelid', 'longitude', 'latitude', 'neighborhood_mean_price']].head(20)

,parcelid,longitude,latitude,neighborhood_mean_price
0,14217523,117760080.0,33834352.0,0.017423
1,11199964,118103008.0,34563684.0,-0.002890
2,12627031,118277552.0,33791928.0,0.017852
3,13992985,117946520.0,33829112.0,0.006387
4,12086693,118232000.0,34139900.0,-0.002875
5,14152642,117926168.0,33936776.0,0.003294
6,17210760,118869328.0,34224248.0,0.018469
7,14145516,117940552.0,33907560.0,0.007479
8,14696645,117833896.0,33608692.0,0.021820
9,11697737,118341384.0,33992188.0,-0.006688


## Colonne aggiuntive calcolate su rapporti ##

In [93]:
# Add three columns to the given dataset dividing other two
def add_tax_info(df):
    df['living_area_prop'] = df['calculatedfinishedsquarefeet'] / df['lotsizesquarefeet']
    df['tax_ratio']        = df['taxvaluedollarcnt']            / df['taxamount']
    df['tax_prop']         = df['structuretaxvaluedollarcnt']   / df['landtaxvaluedollarcnt']
    return df

In [94]:
for df in [X_train, X_val, X_test]:
    df = add_tax_info(df)

## Valori discreti ##

In [95]:
# Given a dataframe and a cut-off returns a list of column-names which has less than cut-off different values
def get_discrete(df, cutoff):
    discretes = []
    for col_name in df.columns:
        values_count = len(get_col(df,col_name).unique())
        if values_count < cutoff:
            discretes.append(col_name)
    return discretes

# Given a list of columns prints for each column all his differente values
def discrete_info(df, discretes):
    for discrete in discretes:
        values = get_col(df, discrete).unique()
        print(f'{discrete}\n{values} ({len(values)})\n')

In [96]:
discrete_info(X_train, get_discrete(X_train, 30))

bathroomcnt
[ 2.5  3.   2.   1.   4.   1.5  5.   3.5  0.   4.5  6.   6.5  5.5  8.
 10.   7.5  7.  15.   9.  11.  18.  13.  12.   8.5] (24)

bedroomcnt
[ 3.  4.  5.  0.  2.  1.  6.  8.  7.  9. 10. 12. 11. 16. 14. 13.] (16)

buildingqualitytypeid
[nan  8.  7.  4.  6. 10.  9.  1.  5. 11. 12.  3.  2.] (13)

calculatedbathnbr
[ 2.5  3.   2.   1.   4.   1.5  5.   3.5  nan  4.5  6.   6.5  5.5  8.
 10.   7.5  7.  15.   9.  11.  18.  13.  12.   8.5] (24)

fips
[6059. 6037. 6111.] (3)

fireplacecnt
[0. 1. 2. 3. 4. 5.] (6)

heatingorsystemtypeid
[nan  2.  7. 24.  6. 13. 20. 18.  1. 11. 10. 12.] (12)

poolcnt
[0. 1.] (2)

propertycountylandusecode
['122' '0100' 'rare' '34' '010C' '0101'] (6)

propertylandusetypeid
[261. 266. 269. 247. 265. 246. 275. 267. 260. 248. 263.  31. 264.] (13)

propertyzoningdesc
['rare' 'LAR1' 'LAR3' 'LARD1.5' 'LBR1N' 'LARS'] (6)

regionidcounty
[1286. 3101. 2061.] (3)

roomcnt
[ 7.  0.  8.  6.  5.  4.  9. 10.  3.  2. 11. 15. 12. 13. 14.  1.] (16)

unitcnt
[ nan   1.   3.

In [97]:
# Valori discreti: cateogirci e ordinali
categorical = ['fips',\
               'heatingorsystemtypeid',\
               'poolcnt',\
               'propertycountylandusecode',\
               'propertylandusetypeid',\
               'propertyzoningdesc',\
               'regionidcounty'\
              ]
ordinal = ['bathroomcnt',\
           'bedroomcnt',\
           'buildingqualitytypeid',\
           'calculatedbathnbr',\
           'fireplacecnt',\
           'roomcnt',\
           'unitcnt'\
          ]

In [98]:
# Valori continui: numerici:
numeric = list(set(X_train.columns) - set(categorical + ordinal) - {'parcelid'})

In [99]:
X_train[numeric].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99709 entries, 0 to 100731
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   lotsizesquarefeet             88847 non-null  float32
 1   latitude                      99709 non-null  float32
 2   structuretaxvaluedollarcnt    99436 non-null  float32
 3   neighborhood_mean_price       99700 non-null  float64
 4   longitude                     99709 non-null  float32
 5   living_area_prop              88583 non-null  float32
 6   tax_prop                      99436 non-null  float32
 7   calculatedfinishedsquarefeet  99224 non-null  float32
 8   regionidcity                  97773 non-null  float32
 9   int_transactiondate           99709 non-null  float64
 10  yearbuilt                     99138 non-null  float32
 11  regionidzip                   99661 non-null  float32
 12  finishedsquarefeet12          94874 non-null  float32
 13  

In [100]:
dimensionality()

X_train (99709, 35)
X_val   (33572, 35)
X_test  (33567, 35)


In [101]:
len(numeric) + len(categorical) + len(ordinal) + 1

35

In [102]:
def dim_check():
    return X_train.shape[1] == len(numeric) + len(categorical) + len(ordinal) + 1

In [103]:
dim_check()

True

In [104]:
print(f'Numeric:\n{numeric} ({len(numeric)})\n')
print(f'Categorical:\n{categorical} ({len(categorical)})\n')
print(f'Ordinal:\n{ordinal} ({len(ordinal)})\n')

Numeric:
['lotsizesquarefeet', 'latitude', 'structuretaxvaluedollarcnt', 'neighborhood_mean_price', 'longitude', 'living_area_prop', 'tax_prop', 'calculatedfinishedsquarefeet', 'regionidcity', 'int_transactiondate', 'yearbuilt', 'regionidzip', 'finishedsquarefeet12', 'taxamount', 'landtaxvaluedollarcnt', 'assessmentyear', 'tax_ratio', 'period_mean_price', 'rawcensustractandblock', 'taxvaluedollarcnt'] (20)

Categorical:
['fips', 'heatingorsystemtypeid', 'poolcnt', 'propertycountylandusecode', 'propertylandusetypeid', 'propertyzoningdesc', 'regionidcounty'] (7)

Ordinal:
['bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'fireplacecnt', 'roomcnt', 'unitcnt'] (7)



## Missing Values: numerici e ordinali ##

Controllo la percentuale di missing value in questi tipi di dati per controllare se è sensato inserire le missing-flags

In [105]:
def over_nan_percentage(colnames, cutoff, verbose=False):
    over = []
    for cn in colnames:
        col = get_col(X_train, cn)
        _, perc = get_col_nan_info(col)
        if verbose:
            print(f'{cn}: {perc}')
        if perc > cutoff:
            over.append(cn)
    return over

In [106]:
put_nan_flag = over_nan_percentage(numeric+ordinal, 0.2, verbose=True)
put_nan_flag

lotsizesquarefeet: 0.10893700668946635
latitude: 0.0
structuretaxvaluedollarcnt: 0.002737967485382463
neighborhood_mean_price: 9.0262664353268e-05
longitude: 0.0
living_area_prop: 0.11158471151049554
tax_prop: 0.002737967485382463
calculatedfinishedsquarefeet: 0.004864154690148332
regionidcity: 0.019416502020880765
int_transactiondate: 0.0
yearbuilt: 0.0057266645939684484
regionidzip: 0.00048140087655076274
finishedsquarefeet12: 0.0484911091275612
taxamount: 0.0001002918492814089
landtaxvaluedollarcnt: 1.002918492814089e-05
assessmentyear: 0.0
tax_ratio: 0.00011032103420954979
period_mean_price: 0.0
rawcensustractandblock: 0.0
taxvaluedollarcnt: 1.002918492814089e-05
bathroomcnt: 0.0
bedroomcnt: 0.0
buildingqualitytypeid: 0.3625650643372213
calculatedbathnbr: 0.010149535147278581
fireplacecnt: 0.0
roomcnt: 0.0
unitcnt: 0.35170345706004474


['buildingqualitytypeid', 'unitcnt']

I missing value hanno una bassissima percentaule, scelgo di non usare i missing-flag, fatta eccezione per unitcnt. <br>
Uso il missing flag per unitcnt

In [107]:
# Given a dataframe and a column_name adds the missing flag
def add_missing_flag(df, col_name):
    df[col_name+'_na_flag'] = df.loc[:,col_name].isna().astype(int)
    return df

In [108]:
for df in [X_train, X_val, X_test]:
    for cname in put_nan_flag:
        df = add_missing_flag(df, cname)

In [109]:
for cname in put_nan_flag:
    print(X_train.loc[:, [cname, cname+'_na_flag']])

        buildingqualitytypeid  buildingqualitytypeid_na_flag
0                         NaN                              1
1                         8.0                              0
2                         7.0                              0
3                         NaN                              1
4                         4.0                              0
...                       ...                            ...
100727                    7.0                              0
100728                    NaN                              1
100729                    NaN                              1
100730                    NaN                              1
100731                    7.0                              0

[99709 rows x 2 columns]
        unitcnt  unitcnt_na_flag
0           NaN                1
1           1.0                0
2           1.0                0
3           NaN                1
4           1.0                0
...         ...              ...
100727     

In [110]:
dimensionality()

X_train (99709, 37)
X_val   (33572, 37)
X_test  (33567, 37)


In [111]:
# Given a dataframe and its column names fill its Nans with the median value of the column for that region
def fill_nan_with_median_same_country(df, col_names, country_ids):   
    for country_id in country_ids:
        df_sub = df[df.loc[:,'regionidcounty'] == country_id]
        df_sub = fill_nan_with_median(df_sub, col_names)
    return df

# Given a dataframe and its column names fill its Nans with the median value of the column
def fill_nan_with_median(df, col_names):   
    for col_name in col_names:
        df[col_name] = df[col_name].fillna(get_col(df, col_name).median())
    return df

In [112]:
for X in [X_train, X_val, X_test]:
    X = fill_nan_with_median(X, numeric+ordinal)
    # X = fill_nan_with_median_same_country(X, numeric+ordinal, [1286., 2061., 3101.])

In [113]:
X_train[numeric + ordinal].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99709 entries, 0 to 100731
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   lotsizesquarefeet             99709 non-null  float32
 1   latitude                      99709 non-null  float32
 2   structuretaxvaluedollarcnt    99709 non-null  float32
 3   neighborhood_mean_price       99709 non-null  float64
 4   longitude                     99709 non-null  float32
 5   living_area_prop              99709 non-null  float32
 6   tax_prop                      99709 non-null  float32
 7   calculatedfinishedsquarefeet  99709 non-null  float32
 8   regionidcity                  99709 non-null  float32
 9   int_transactiondate           99709 non-null  float64
 10  yearbuilt                     99709 non-null  float32
 11  regionidzip                   99709 non-null  float32
 12  finishedsquarefeet12          99709 non-null  float32
 13  

## One-hot encoding delle variabili categoriali ##

In [114]:
categorical

['fips',
 'heatingorsystemtypeid',
 'poolcnt',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'regionidcounty']

In [115]:
# Given a train-dataframe, its column-names and a list of dataframes,
# trains a one-hot-encoder to the train
# makes a one-hot-enconding for each dataframe
def one_hot_encoding(df_fit, col_names, dfs):
    oh = OneHotEncoder(sparse=False, handle_unknown='ignore')
    oh.fit(df_fit[col_names])
    for df in dfs:
        encoded = oh.transform(df[col_names])
        for i, col in enumerate(oh.get_feature_names(col_names)):
            df[col] = encoded[:,i]
        df.drop(col_names, axis=1, inplace=True)

In [116]:
one_hot_encoding(X_train, categorical, [X_train, X_val, X_test])

In [117]:
dimensionality()

X_train (99709, 75)
X_val   (33572, 75)
X_test  (33567, 75)


Rimuovo colonne che codficano i Nan per One-Hot-Encoding: mantengo righe di soli zeri

In [118]:
nan_column = list(filter(re.compile("^.*_nan$").match, list(X_train.columns)))
print(nan_column)

['heatingorsystemtypeid_nan']


In [119]:
for X in [X_train, X_val, X_test]:
    X = remove_column(X, nan_column)
    X = remove_column(X, 'poolcnt_0.0')

Fireplace flag e poolcnt sono variabili binarie

In [120]:
dimensionality()

X_train (99709, 73)
X_val   (33572, 73)
X_test  (33567, 73)


# Scrittura csv #

In [122]:
dir_name = 'preparazione'

X_train.to_csv(dir_name + '/X_train.csv', index=False)
y_train.to_csv(dir_name + '/y_train.csv', index=False)
X_val  .to_csv(dir_name + '/X_val.csv',   index=False)
y_val  .to_csv(dir_name + '/y_val.csv',   index=False)
X_test .to_csv(dir_name + '/X_test.csv',  index=False)
y_test .to_csv(dir_name + '/y_test.csv',  index=False)